# Challenge 2 - Deep Learning

In this tutorial, you will train a PyTorch model on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using distributed training via Nccl/Gloo across a GPU cluster. 

## 1. Import Azure ML Python Python SDK

In [1]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

WARNING - Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception RunType azureml.scriptrun already has a factory, can't add another.


SDK version: 1.0.83


## 2. Authentication and initializing Azure Machine Learning Workspace

As a first step you have to authenticate against the Azure [Machine Learning Workspace](https://ml.azure.com/). This can be achieved in different ways:

1. **Interactive Login Authentication:** The interactive authentication is suitable for local experimentation on your own computer.
2. **Azure CLI Authentication:** Azure CLI authentication is suitable if you are already using Azure CLI for managing Azure resources, and want to sign in only once.
3. **Managed Service Identity (MSI) Authentication:** The MSI authentication is suitable for automated workflows, for example as part of Azure Devops build.
4. **Service Principal Authentication:** The Service Principal authentication is suitable for automated workflows, for example as part of Azure Devops build.

For now, we will use the interactive authentication, which is the default mode when using Azure ML SDK. When you connect to your workspace using `Workspace.from_config`, you will get an interactive login dialog.

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location, 
      "Subscription id: " + ws.subscription_id, 
      "Resource group: " + ws.resource_group, sep = '\n')

Workspace name: myworkspace
Azure region: westeurope
Subscription id: 558bd446-4212-46a2-908c-9ab0a628705e
Resource group: azure-ml-service-rg


## 3. Upload and register data

Every workspace comes with a default [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data) (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and create Dataset from it. We will now upload the Iris data to the default datastore (blob) within your workspace.

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [ ]:
# List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

For this challenge we will use the [default datastore](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#get-datastores-from-your-workspace) that comes with the Azure Machine Learning Workspace.

In [ ]:
# get the default datastore
datastore = ws.get_default_datastore()
print(datastore.name, datastore.datastore_type, datastore.account_name, datastore.container_name, sep="\n")

In [ ]:
datastore.upload_files(files = ['./train-dataset/iris.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

Then we will create and register a TabularDataset pointing to the path in the datastore. You can also create a Dataset from multiple paths. [Learn more](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets)

A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. Dataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [ ]:
from azureml.core import Dataset

tabular_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/iris.csv')])
tabular_dataset = tabular_dataset.register(workspace=ws,
                                           name='iris_tabular',
                                           description='tabular iris training data',
                                           create_new_version = True)

# preview the first 3 rows of the dataset
tabular_dataset.take(3).to_pandas_dataframe()

Now we will register a dataset in the Azure Machine Learning Workspace as a file dataset. A file dataset can be mounted to the compute engine. When you mount a file system, you attach that file system to a directory (mount point) and make it available to the system. Because mounting load files at the time of processing, it is usually faster than download.
Note: mounting is only available for Linux-based compute (DSVM/VM, AMLCompute, HDInsights).

In [ ]:
from azureml.core import Dataset

file_dataset = Dataset.File.from_files(path = [(datastore, 'train-dataset/tabular/iris.csv')])
file_dataset = file_dataset.register(workspace=ws,
                                     name='iris_file',
                                     description='file iris training data',
                                     create_new_version = True)

file_dataset.to_path()

## 3. Create Compute Engine

In this sample, we want to train a simple scikit-learn model on a remote compute engine on Azure. To do so, we first must create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target).

In this challenge, we want to use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Once this is created, you are ready to train on your remote compute.

#### **Task:** Create a machine learning compute target.

Create an Azure Machine Learning Compute cluster and folow the steps one to four.
1. Check whether the cluster with the given name already exists.
2. Create the configuration (this step is local and only takes a second). Use the SKU `STANDARD_D2_V2` and a maximum of 4 nodes.
3. Create the cluster (this step will take about 20 seconds)
4. Provision the VMs to bring the cluster to the initial size. This step will take about 3-5 minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-10-24T09:30:26.323000+00:00', 'errors': None, 'creationTime': '2019-02-12T12:37:37.681874+00:00', 'modifiedTime': '2019-02-12T12:38:17.800043+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## 4. Create a project directory 

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [5]:
TRAIN_FOLDER_NAME = 'train'
TRAIN_FILE_NAME = 'train.py'

In [6]:
import os
os.makedirs(os.path.join(".", TRAIN_FOLDER_NAME), exist_ok=True)

## 5. Create a training script 

Now you will need to create your training scripts in your project folder. This will be done in the next step. In practice, you should be able to take any custom training script as is and run it with Azure ML without having to modify your code.

If you would like to use Azure ML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of Azure ML code inside your training script.

In `train_iris.py`, we will log some metrics to our Azure ML run. To do so, we will access the Azure ML Run object within the script:

```python
from azureml.core.run import Run
run = Run.get_context()
```

Further within `train_iris.py`, we log the kernel and penalty parameters, and the highest accuracy the model achieves:

```python
run.log('Kernel type', np.string(args.kernel))
run.log('Penalty', np.float(args.penalty))

run.log('Accuracy', np.float(accuracy))
```

These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section.

In [24]:
%%writefile $TRAIN_FOLDER_NAME/$TRAIN_FILE_NAME

from __future__ import print_function
import argparse
import os
import shutil
import time
import onnx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
import torchvision.models as models

from azureml.core.run import Run
# get the Azure ML run object
run = Run.get_context()

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--world-size', default=1, type=int,
                    help='number of distributed processes')
parser.add_argument('--dist-url', type=str,
                    help='url used to set up distributed training')
parser.add_argument('--dist-backend', default='nccl', type=str,
                    help='distributed backend')
parser.add_argument('--rank', default=-1, type=int,
                    help='rank of the worker')

best_prec1 = 0
args = parser.parse_args()

args.distributed = args.world_size >= 2

if args.distributed:
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                            world_size=args.world_size, rank=args.rank)

train_dataset = datasets.MNIST('data-%d' % args.rank, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

if args.distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
else:
    train_sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, shuffle=(train_sampler is None),
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)


test_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


model = Net()

if not args.distributed:
    model = torch.nn.DataParallel(model).cuda()
else:
    model.cuda()
    model = torch.nn.parallel.DistributedDataParallel(model)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.momentum, weight_decay=args.weight_decay)


def train(epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()
    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        try:
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(prec1[0], input.size(0))
            top5.update(prec5[0], input.size(0))

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 5 == 0:
                run.log("loss", losses.avg)
                run.log("prec@1", "{0:.3f}".format(top1.avg))
                run.log("prec@5", "{0:.3f}".format(top5.avg))
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(epoch, i, len(train_loader),
                                                                      batch_time=batch_time, data_time=data_time,
                                                                      loss=losses, top1=top1, top5=top5))
        except:
            import sys
            print("Unexpected error:", sys.exc_info()[0])


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


for epoch in range(1, args.epochs + 1):
    train(epoch)

# Save model as pt file
torch.save(model, os.path.join('outputs', 'model.pt'))

# Save model as ONNX file
dummy_input = torch.randn(1, 1, 28, 28, device='cuda')
torch.onnx.export(model,
                  dummy_input,
                  os.path.join('outputs', 'model.onnx'),
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  verbose=True,
                  input_names=[ "image" ],
                  output_names=[ "log_softmax_pred" ])

# Check ONNX model
onnx_model = onnx.load(os.path.join('outputs', 'model.onnx'))
onnx.checker.check_model(onnx_model)

Overwriting train/train.py


## 6. Create an experiment

An *Experiment* is a logical container in an Azure ML Workspace that represents a collection of trials (individual model runs). It hosts run records which can include run metrics and output artifacts from your experiments.

In [25]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='pytorch_sample')

## 7. Create Estimator

An estimator object is used to submit the run. Azure Machine Learning has pre-configured estimators for common machine learning frameworks, as well as generic Estimator. Create a generic estimator for by specifying

- The name of the estimator object, est
- The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution.
- The training script name, train_titanic.py
- The input Dataset for training
- The compute target. In this case you will use the AmlCompute you created
- The environment definition for the experiment

In [26]:
from azureml.train.dnn import PyTorch, Nccl

script_params = {'--dist-backend' : 'nccl',
                 '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
                 '--rank': '$AZ_BATCHAI_TASK_INDEX',
                 '--world-size': 2,
                 '--epochs': 1}

est = PyTorch(source_directory=TRAIN_FOLDER_NAME,
              entry_script=TRAIN_FILE_NAME,
              script_params=script_params,
              compute_target=compute_target,
              node_count=2,
              distributed_training=Nccl(),
              use_gpu=True,
              framework_version='1.3') 

In the above code, `script_params` uses Azure ML generated `AZ_BATCHAI_PYTORCH_INIT_METHOD` for shared file-system initialization and `AZ_BATCHAI_TASK_INDEX` as rank of each worker process.
The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using Nccl, you must provide the argument `distributed_training=Nccl()`. Using this estimator with these settings, PyTorch and dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters.

In [27]:
'''
# Alternative
from azureml.train.dnn import PyTorch, Gloo

script_params = {'--dist-backend' : 'gloo',
                 '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
                 '--rank': '$AZ_BATCHAI_TASK_INDEX',
                 '--world-size': 2}

est = PyTorch(source_directory=TRAIN_FOLDER_NAME,
                    entry_script=TRAIN_FILE_NAME,
                    script_params=script_params,
                    compute_target=compute_target,
                    node_count=2,
                    distributed_training=Gloo(),
                    use_gpu=True)
'''

"\n# Alternative\nfrom azureml.train.dnn import PyTorch, Gloo\n\nscript_params = {'--dist-backend' : 'gloo',\n                 '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',\n                 '--rank': '$AZ_BATCHAI_TASK_INDEX',\n                 '--world-size': 2}\n\nest = PyTorch(source_directory=TRAIN_FOLDER_NAME,\n                    entry_script=TRAIN_FILE_NAME,\n                    script_params=script_params,\n                    compute_target=compute_target,\n                    node_count=2,\n                    distributed_training=Gloo(),\n                    use_gpu=True)\n"

In the above code, `script_params` uses Azure ML generated `AZ_BATCHAI_PYTORCH_INIT_METHOD` for shared file-system initialization and `AZ_BATCHAI_TASK_INDEX` as rank of each worker process.
The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using Gloo, you must provide the argument `distributed_training=Gloo()`. Using this estimator with these settings, PyTorch and dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters.

Once you create the estimaotr you can follow the submit steps as shown above to submit a PyTorch run with `Gloo` backend. 

## 8. Submit the job

Submit the estimator to the Azure ML experiment to kick off the execution.

In [28]:
run = exp.submit(est)
run.wait_for_completion(show_output=True, wait_post_processing=True)

RunId: pytorch_sample_1578647429_5ab88482
Web View: https://ml.azure.com/experiments/pytorch_sample/runs/pytorch_sample_1578647429_5ab88482?wsid=/subscriptions/558bd446-4212-46a2-908c-9ab0a628705e/resourcegroups/azure-ml-service-rg/workspaces/myworkspace

Streaming azureml-logs/55_azureml-execution-tvmps_9560b8e3c57583a99c08e3cfd5df49a273a054ed3867157a53d5fe8f75ec36fc_d.txt

2020-01-10T09:14:58Z Starting output-watcher...
Login Succeeded
1.3-gpu: Pulling from pytorch
35c102085707: Pulling fs layer
251f5509d51d: Pulling fs layer
8e829fe70a46: Pulling fs layer
6001e1789921: Pulling fs layer
9f0a21d58e5d: Pulling fs layer
47b91ac70c27: Pulling fs layer
a0529eb74f28: Pulling fs layer
23bff6dcced5: Pulling fs layer
2137cd2bcba9: Pulling fs layer
534a428e13b2: Pulling fs layer
9489ba490035: Pulling fs layer
d52114015b68: Pulling fs layer
4ad1a9a2eefc: Pulling fs layer
73fda3cf0506: Pulling fs layer
58df80ca3f37: Pulling fs layer
e2f521c73c2d: Pulling fs layer
4ed3983585cc: Pulling fs layer
8

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ModuleNotFoundError: No module named 'onnx'",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'onnx'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/myworkspace/azureml/pytorch_sample_1578647429_5ab88482/mounts/workspaceblobstore/azureml/pytorch_sample_1578647429_5ab88482/azureml-setup/context_manager_injector.py\", line 119, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/opt/miniconda/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/opt/miniconda/lib/python3.6/runpy.py\", line 96, in _run_module_code\n    mod_name, mod_spec, pkg_name, script_name)\

In [17]:
#run.cancel()

NameError: name 'run' is not defined

You now have a model trained on a remote cluster. Retrieve all the metrics logged during the run, including the accuracy of the model:

In [ ]:
run.get_metrics()

## 9. Tune model hyperparameters

Now that we've seen how to do a simple Scikit-learn training run using the SDK, let's see if we can further improve the accuracy of our model. We can optimize our model's hyperparameters using Azure Machine Learning's hyperparameter tuning capabilities.

First, we will define the hyperparameter space to sweep over. Let's tune the `kernel` and `penalty` parameters. In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, `Accuracy`.

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import BayesianParameterSampling, RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform
from azureml.train.hyperdrive.policy import BanditPolicy

param_sampling = RandomParameterSampling({
    '--dist-backend' : 'nccl',
    '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
    '--rank': '$AZ_BATCHAI_TASK_INDEX',
    '--world-size': 2,
    '--lr': loguniform(0.0005, 0.01),
    '--momentum': uniform(0.45, 0.55),
    '--weight-decay': uniform(1e-5, 1e-3)
    })

hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='loss',
                                         primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                         max_total_runs=16,
                                         max_concurrent_runs=4,
                                         policy=BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=1))

Finally, lauch the hyperparameter tuning job.

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True, wait_post_processing=True)

Often times, finding the best hyperparameter values for your model can be an iterative process, needing multiple tuning runs that learn from previous hyperparameter tuning runs. Reusing knowledge from these previous runs will accelerate the hyperparameter tuning process, thereby reducing the cost of tuning the model and will potentially improve the primary metric of the resulting model. When warm starting a hyperparameter tuning experiment with Bayesian sampling, trials from the previous run will be used as prior knowledge to intelligently pick new samples, so as to improve the primary metric. Additionally, when using Random or Grid sampling, any early termination decisions will leverage metrics from the previous runs to determine poorly performing training runs. 

Azure Machine Learning allows you to warm start your hyperparameter tuning run by leveraging knowledge from up to 5 previously completed hyperparameter tuning parent runs. 

Additionally, there might be occasions when individual training runs of a hyperparameter tuning experiment are cancelled due to budget constraints or fail due to other reasons. It is now possible to resume such individual training runs from the last checkpoint (assuming your training script handles checkpoints). Resuming an individual training run will use the same hyperparameter configuration and mount the storage used for that run. The training script should accept the "--resume-from" argument, which contains the checkpoint or model files from which to resume the training run. You can also resume individual runs as part of an experiment that spends additional budget on hyperparameter tuning. Any additional budget, after resuming the specified training runs is used for exploring additional configurations.

For more information on warm starting and resuming hyperparameter tuning runs, please refer to the [Hyperparameter Tuning for Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) 

When all jobs finish, we can find out the one that has the highest accuracy.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

## 9. Register model

The last step in the training script wrote the file `model.pkl` in a directory named `outputs` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can see files associated with that run.

In [ ]:
best_run.get_file_names()

Register the model in the workspace so that you (or other collaborators) can later query, examine, and deploy this model.

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = best_run.register_model(model_name='pytorch-model',
                                model_path='outputs/model.pt',
                                description='PyTorch MNIST classification.',
                                tags={'area': 'mnist', 'type': 'pytorch'})

print(model.name, model.id, model.version, sep='\n')

Now, your model is ready for deployment.

## 10. Deployment

No-code model deployment is currently in preview and supports various frameworks and model types including Tensorflow SavedModel format, ONNX models and Scikit-learn models. No code model deployment is supported for all built-in scikit-learn model types.

The deployment will take a few minutes and will take place on an Azure Container Instance.

In [ ]:
service_no_code = Model.deploy(workspace=ws,
                               name='sample-service',
                               models=[model])
service_no_code.wait_for_deployment(show_output=True)

In [ ]:
# If deployment fails, then retry with:
# service_no_code.update(models=[model])

In [ ]:
service_no_code.get_logs()

Convert this Webservice object into a JSON serialized dictionary, which lists all the details of the webservice.

In [ ]:
service_no_code.serialize()

## 11. Test Service

The following code is an example of a Python client that can be used with the container.

In [ ]:
import json

# Two sets of data to score, so we get two results back.
data = {'data':
        [
            [ 1,2,3,4 ],
            [ 10,9,8,7 ]
        ]
        }
# Convert to JSON string.
input_data = json.dumps(data)

# Make the request and display the response.
resp = service_no_code.run(input_data)
print(resp)

Delete the service to save cost.

In [ ]:
service_no_code.delete()

## 11. Only Package models

In some cases, you might want to create a Docker image without deploying the model (if, for example, you plan to deploy to Azure App Service). Or you might want to download the image and run it on a local Docker installation. You might even want to download the files used to build the image, inspect them, modify them, and build the image manually.

Model packaging enables you to do these things. It packages all the assets needed to host a model as a web service and allows you to download either a fully built Docker image or the files needed to build one. There are two ways to use model packaging:

Download a packaged model: Download a Docker image that contains the model and other files needed to host it as a web service.

Generate a Dockerfile: Download the Dockerfile, model, entry script, and other assets needed to build a Docker image. You can then inspect the files or make changes before you build the image locally.

Both packages can be used to get a local Docker image.

In [ ]:
SCORE_FOLDER_NAME = 'deployment'
SCORE_FILE_NAME = 'score.py'

In [ ]:
import os
os.makedirs(os.path.join(".", SCORE_FOLDER_NAME), exist_ok=True)

In [ ]:
%%writefile $SCORE_FOLDER_NAME/$SCORE_FILE_NAME

import joblib
import numpy as np
import os

from azureml.monitoring import ModelDataCollector
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType

# The init() method is called once, when the web service starts up.
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model
    global inputs_dc, prediction_dc
    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'sample-model'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)
    model = joblib.load(model_path)
    inputs_dc = ModelDataCollector("sample-model", designation="inputs", feature_names=["feat1", "feat2", "feat3", "feat4"])
    prediction_dc = ModelDataCollector("sample-model", designation="predictions", feature_names=["prediction"])

# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example input you provide here. This will also generate a Swagger
# API document for your web service.
@input_schema('data', NumpyParameterType(np.array([[0.1, 1.2, 2.3, 3.4]])))
@output_schema('predict', StandardPythonParameterType([['Iris-virginica']]))
def run(data):
    # Use the model object loaded by init().
    result = { 'predict': model.predict(data) }

    # You can return any JSON-serializable object.
    return result

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment(name='my-sample-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'joblib',
    'numpy',
    'scikit-learn',
    'inference-schema',
    'inference-schema[numpy-support]',
    'azureml-monitoring'
])
environment.register(workspace=ws)

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script=SCORE_FILE_NAME,
                                   source_directory=SCORE_FOLDER_NAME,
                                   description='SVC classification for iris dataset.',
                                   environment=environment)

In [ ]:
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)

In [ ]:
acr = package.get_container_registry()
print("Address:", acr.address)

Pull the package output to the local machine. This can only be used with a Docker image package. Docker must be running on the machine.

In [ ]:
package.pull()

## 12. Deployment with inference_config

In [ ]:
service = Model.deploy(workspace=ws,
                       name='sample-service-new',
                       models=[model],
                       inference_config=inference_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
# If deployment fails, then retry with:
# service.update(models=[model], inference_config=inference_config)

In [ ]:
service.get_logs()

In [ ]:
service.serialize()

In [ ]:
import json

# Two sets of data to score, so we get two results back.
data = {'data':
        [
            [ 1,2,3,4 ],
            [ 10,9,8,7 ]
        ]
        }
# Convert to JSON string.
input_data = json.dumps(data)

# Make the request and display the response.
resp = service.run(input_data)
print(resp)

In [ ]:
service.delete()